# This notebook simulates the di-lepton decays of HNLs


In [1]:
%load_ext autoreload
%autoreload 2

In [110]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
from matplotlib.pyplot import *
from matplotlib.legend_handler import HandlerLine2D

from HNL_MC import *
from HNL_MC import hnl_tools
from HNL_MC.const import *
from HNL_MC import nuH_gen as gen
from HNL_MC.plot_style import *


### Generate and load events

In [3]:
PATH = 'data/MC_samples'
hels = ['LH','RH', 'both']
vertices = ['CConly', 'NConly', 'CCandNC']
hnltypes = ['majorana','dirac']
hnlmasses = [0.02,0.05,0.1,0.150]
my_mixing = [0.0,1.0,0.0]
dfs = {}
for h in hels:
    for vertex in vertices:
        for hnltype in hnltypes:
            for hnlmass in hnlmasses:
                gen.generate_events(hnlmass, mixings=my_mixing, lepton_mass=m_e, 
                                    HNLtype=hnltype, HEL=h, 
                                    modify_vertex=vertex)
                dfs[f'{hnlmass}_ee_{h}_{hnltype}_{vertex}'] = pd.read_pickle(f'{PATH}/MC_m4_{hnlmass:.8g}_mlepton_{m_e:.8g}_hel_{h}_{hnltype}_{vertex}.pckl')
                

data/mc_samples/MC_m4_0.02_mlepton_0.00051099895_hel_LH_majorana_CConly.pckl
data/mc_samples/MC_m4_0.05_mlepton_0.00051099895_hel_LH_majorana_CConly.pckl
data/mc_samples/MC_m4_0.1_mlepton_0.00051099895_hel_LH_majorana_CConly.pckl
data/mc_samples/MC_m4_0.15_mlepton_0.00051099895_hel_LH_majorana_CConly.pckl
data/mc_samples/MC_m4_0.02_mlepton_0.00051099895_hel_LH_dirac_CConly.pckl
data/mc_samples/MC_m4_0.05_mlepton_0.00051099895_hel_LH_dirac_CConly.pckl
data/mc_samples/MC_m4_0.1_mlepton_0.00051099895_hel_LH_dirac_CConly.pckl
data/mc_samples/MC_m4_0.15_mlepton_0.00051099895_hel_LH_dirac_CConly.pckl
data/mc_samples/MC_m4_0.02_mlepton_0.00051099895_hel_LH_majorana_NConly.pckl
data/mc_samples/MC_m4_0.05_mlepton_0.00051099895_hel_LH_majorana_NConly.pckl
data/mc_samples/MC_m4_0.1_mlepton_0.00051099895_hel_LH_majorana_NConly.pckl
data/mc_samples/MC_m4_0.15_mlepton_0.00051099895_hel_LH_majorana_NConly.pckl
data/mc_samples/MC_m4_0.02_mlepton_0.00051099895_hel_LH_dirac_NConly.pckl
data/mc_samples/M

#### compute kinematical variables

In [174]:
# compute useful variables
for key in dfs.keys():
    gen.compute_kin_vars(dfs[key])

### Plotting all variables for varying m4 values

In [232]:
my_list_of_colors = ['black','violet','darkorange','royalblue']
my_list_of_dashes = ['-','-','-','-']
my_hatches = ['/////////////////////////////',
              '/////////////////////////////',
              '/////////////////////////////',
              '/////////////////////////////']

def plot_varying_m4(var, MIN,MAX, hnltype='majorana', hel='LH', vertex='NConly', xlabel='x', colors=my_list_of_colors, units = 1, cum=''):

    fig,ax = get_fig()
    
    for i in range(len(hnlmasses)):
        mN = hnlmasses[i]
#         my_histogram(ax, dfs[f'{mN}_ee_{hel}_{hnltype}_{vertex}'], var, var_range=(MIN, MAX), units = units,
#                      color=colors[i],label=fr'$m_N = {mN*1e3}$ MeV', density=True, ls=my_list_of_dashes[i], hatch=my_hatches[i])
        error_histogram(ax, dfs[f'{mN}_ee_{hel}_{hnltype}_{vertex}'], var, var_range=(MIN, MAX), units = units,
                         color=colors[i],label=fr'$m_N = {mN*1e3}$ MeV', 
                            density=True, ls=my_list_of_dashes[i], hatch=my_hatches[i],
                           cumulative=cum)

    ax.patch.set_alpha(0.0)

    
    ax.legend(loc='upper right',frameon=False,ncol=1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(r"a.u.")
    ax.set_xlim(MIN,MAX)
    ax.set_ylim(0.,ax.get_ylim()[1]*1.1)
    # ax.set_yscale('log')
    ax.set_title(fr"$N \to \nu e^+e^-$ $-$ {hnltype} {hel} {vertex}",fontsize=fsize)
    filename=f'{hnltype}/{vertex}/{cum}/{hnltype}_{hel}_{vertex}_{var}'
    print(filename)
    plt.savefig(f"plots/kinematics/{filename}.png", dpi=400)
    plt.close()

def all_vars(cum='', hnltype='majorana', hel='LH', vertex='NConly'):
    plot_varying_m4('easy',-1,1, xlabel=r'$(E_{e^+} - E_{e^-})/(E_{e^+} + E_{e^-})$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('ee_momentum',0,7, xlabel=r'$E_{ee}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('e_smallest',0,7, xlabel=r'$E_{e}^{\rm min}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('e_largest',0,7, xlabel=r'$E_{e}^{\rm max}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('ee_theta',0,20, units = 180/np.pi, xlabel=r'$\Delta \theta_{ee}\,(^\circ)$', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('ee_beam_theta',0,10, units = 180/np.pi, xlabel=r'$\theta_{ee}^{\rm beam} \, (^\circ)$', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('ee_beam_costheta',0.998,1, units = 1, xlabel=r'$\cos(\theta_{ee}^{\rm beam})$', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('radius_plus',0,100, units = 1e-2, xlabel=r'curvature radius (m)', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('distance_between_circles_at_10cm',0,3, units = 1, xlabel=r'distance between bent trajectories at 10 cm', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('distance_between_circles_at_20cm',0,6, units = 1, xlabel=r'distance between bent trajectories at 20 cm', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('distance_between_circles_at_50cm',0,20, units = 1, xlabel=r'distance between bent trajectories at 50 cm', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
    plot_varying_m4('race_to_b=1cm',0,10, units = 1e-2, xlabel=r'travel distance to b=1 (cm)', 
                    cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)

cum_types = ['', 'cum_sum', 'cum_sum_prior_to']
def batch_plot_kinematics():
    for cum in cum_types:
        for hnltype in hnltypes:
            for hel in ['LH','RH']:
                for vertex in vertices:
                    all_vars(cum=cum, hnltype=hnltype, hel=hel, vertex=vertex)
        

In [233]:
import warnings
warnings.filterwarnings('ignore')

In [234]:
batch_plot_kinematics()

majorana/CConly//majorana_LH_CConly_easy
majorana/CConly//majorana_LH_CConly_ee_momentum
majorana/CConly//majorana_LH_CConly_e_smallest
majorana/CConly//majorana_LH_CConly_e_largest
majorana/CConly//majorana_LH_CConly_ee_theta
majorana/CConly//majorana_LH_CConly_ee_beam_theta
majorana/NConly//majorana_LH_NConly_ee_beam_costheta
majorana/CConly//majorana_LH_CConly_radius_plus
majorana/CConly//majorana_LH_CConly_distance_between_circles_at_10cm
majorana/CConly//majorana_LH_CConly_distance_between_circles_at_20cm
majorana/CConly//majorana_LH_CConly_distance_between_circles_at_50cm
majorana/CConly//majorana_LH_CConly_race_to_b=1cm
majorana/NConly//majorana_LH_NConly_easy
majorana/NConly//majorana_LH_NConly_ee_momentum
majorana/NConly//majorana_LH_NConly_e_smallest
majorana/NConly//majorana_LH_NConly_e_largest
majorana/NConly//majorana_LH_NConly_ee_theta
majorana/NConly//majorana_LH_NConly_ee_beam_theta
majorana/NConly//majorana_LH_NConly_ee_beam_costheta
majorana/NConly//majorana_LH_NConly

majorana/NConly/cum_sum/majorana_LH_NConly_ee_theta
majorana/NConly/cum_sum/majorana_LH_NConly_ee_beam_theta
majorana/NConly/cum_sum/majorana_LH_NConly_ee_beam_costheta
majorana/NConly/cum_sum/majorana_LH_NConly_radius_plus
majorana/NConly/cum_sum/majorana_LH_NConly_distance_between_circles_at_10cm
majorana/NConly/cum_sum/majorana_LH_NConly_distance_between_circles_at_20cm
majorana/NConly/cum_sum/majorana_LH_NConly_distance_between_circles_at_50cm
majorana/NConly/cum_sum/majorana_LH_NConly_race_to_b=1cm
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_easy
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_ee_momentum
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_e_smallest
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_e_largest
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_ee_theta
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_ee_beam_theta
majorana/NConly/cum_sum/majorana_LH_NConly_ee_beam_costheta
majorana/CCandNC/cum_sum/majorana_LH_CCandNC_radius_plus
majorana/CCandNC/cum_sum/majorana_LH_CCandN

majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_easy
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_ee_momentum
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_e_smallest
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_e_largest
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_ee_theta
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_ee_beam_theta
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_ee_beam_costheta
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_radius_plus
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_distance_between_circles_at_10cm
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_distance_between_circles_at_20cm
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_distance_between_circles_at_50cm
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_race_to_b=1cm
majorana/CCandNC/cum_sum_prior_to/majorana_LH_CCandNC_easy
majorana/CCandNC/cum_sum_prior_to/majorana_LH_CCandNC_ee_momentum
majorana/CCandNC/cum_sum_prior_to/majorana_LH_CC

dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_ee_theta
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_ee_beam_theta
majorana/NConly/cum_sum_prior_to/majorana_LH_NConly_ee_beam_costheta
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_radius_plus
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_distance_between_circles_at_10cm
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_distance_between_circles_at_20cm
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_distance_between_circles_at_50cm
dirac/CCandNC/cum_sum_prior_to/dirac_RH_CCandNC_race_to_b=1cm


In [239]:
from pathlib import Path

In [243]:
my_list_of_colors = ['black','royalblue','darkorange']
my_list_of_dashes = ['-','-','-']
my_hatches = ['/////////////////////////////',
              '/////////////////////////////',
              '/////////////////////////////']

def plot_varying_vertex(var, MIN, MAX, hnltype='majorana', hel='LH', mN=0.150, xlabel='x', colors=my_list_of_colors, units = 1, cum=''):

    fig,ax = get_fig()
    
    for i in range(len(vertices)):
        vertex = vertices[i]
        
        error_histogram(ax, dfs[f'{mN}_ee_{hel}_{hnltype}_{vertex}'], var, var_range=(MIN, MAX), units = units,
                         color=colors[i],label=fr'{vertex}', 
                            density=True, ls=my_list_of_dashes[i], hatch=my_hatches[i],
                           cumulative=cum)

    ax.patch.set_alpha(0.0)
    ax.legend(loc='upper right',frameon=False,ncol=1)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(r"a.u.")
    ax.set_xlim(MIN,MAX)
    ax.set_ylim(0.,ax.get_ylim()[1]*1.1)
    # ax.set_yscale('log')
    ax.set_title(fr"$N \to \nu e^+e^-$ $-$ {hnltype} {hel} $m_N = ${mN*1e3} MeV",fontsize=fsize)
    filename=f'{hnltype}_{hel}_{mN:.3g}_{var}'
    print(filename)
    path=f"plots/kinematics/{hnltype}/{mN:.3g}/{cum}/"
    Path(path).mkdir(parents=True, exist_ok=True)
    plt.savefig(f'{path}/{filename}.png', dpi=400)
    plt.close()

def all_vars_vertices(cum='', hnltype='majorana', hel='LH', mN=0.150):
    plot_varying_vertex('easy',-1,1, xlabel=r'$(E_{e^+} - E_{e^-})/(E_{e^+} + E_{e^-})$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('ee_momentum',0,7, xlabel=r'$E_{ee}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('e_smallest',0,7, xlabel=r'$E_{e}^{\rm min}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('e_largest',0,7, xlabel=r'$E_{e}^{\rm max}/$GeV', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('ee_theta',0,20, units = 180/np.pi, xlabel=r'$\Delta \theta_{ee}\,(^\circ)$', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('ee_beam_theta',0,10, units = 180/np.pi, xlabel=r'$\theta_{ee}^{\rm beam} \, (^\circ)$', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('ee_beam_costheta',0.998,1, units = 1, xlabel=r'$\cos(\theta_{ee}^{\rm beam})$', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('radius_plus',0,100, units = 1e-2, xlabel=r'curvature radius (m)', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)
    plot_varying_vertex('distance_between_circles_at_10cm',0,3, units = 1, xlabel=r'distance between bent trajectories at 10 cm', 
                    cum=cum, hnltype=hnltype, hel=hel, mN=mN)

cum_types = ['', 'cum_sum', 'cum_sum_prior_to']
def batch_plot_kinematics_vertices():
    for cum in cum_types:
        for hnltype in hnltypes:
            for hel in ['LH','RH']:
                for hnlmass in hnlmasses:
                    all_vars_vertices(cum=cum, hnltype=hnltype, hel=hel, mN=hnlmass)
                    
                    
                    
                    

In [244]:
batch_plot_kinematics_vertices()

majorana_LH_0.02_easy
majorana_LH_0.02_ee_momentum
majorana_LH_0.02_e_smallest
majorana_LH_0.02_e_largest
majorana_LH_0.02_ee_theta
majorana_LH_0.02_ee_beam_theta
majorana_LH_0.02_ee_beam_costheta
majorana_LH_0.02_radius_plus
majorana_LH_0.02_distance_between_circles_at_10cm
majorana_LH_0.05_easy
majorana_LH_0.05_ee_momentum
majorana_LH_0.05_e_smallest
majorana_LH_0.05_e_largest
majorana_LH_0.05_ee_theta
majorana_LH_0.05_ee_beam_theta
majorana_LH_0.05_ee_beam_costheta
majorana_LH_0.05_radius_plus
majorana_LH_0.05_distance_between_circles_at_10cm
majorana_LH_0.1_easy
majorana_LH_0.1_ee_momentum
majorana_LH_0.1_e_smallest
majorana_LH_0.1_e_largest
majorana_LH_0.1_ee_theta
majorana_LH_0.1_ee_beam_theta
majorana_LH_0.1_ee_beam_costheta
majorana_LH_0.1_radius_plus
majorana_LH_0.1_distance_between_circles_at_10cm
majorana_LH_0.15_easy
majorana_LH_0.15_ee_momentum
majorana_LH_0.15_e_smallest
majorana_LH_0.15_e_largest
majorana_LH_0.15_ee_theta
majorana_LH_0.15_ee_beam_theta
majorana_LH_0.15_e

dirac_RH_0.15_ee_beam_theta
dirac_RH_0.15_ee_beam_costheta
dirac_RH_0.15_radius_plus
dirac_RH_0.15_distance_between_circles_at_10cm
majorana_LH_0.02_easy
majorana_LH_0.02_ee_momentum
majorana_LH_0.02_e_smallest
majorana_LH_0.02_e_largest
majorana_LH_0.02_ee_theta
majorana_LH_0.02_ee_beam_theta
majorana_LH_0.02_ee_beam_costheta
majorana_LH_0.02_radius_plus
majorana_LH_0.02_distance_between_circles_at_10cm
majorana_LH_0.05_easy
majorana_LH_0.05_ee_momentum
majorana_LH_0.05_e_smallest
majorana_LH_0.05_e_largest
majorana_LH_0.05_ee_theta
majorana_LH_0.05_ee_beam_theta
majorana_LH_0.05_ee_beam_costheta
majorana_LH_0.05_radius_plus
majorana_LH_0.05_distance_between_circles_at_10cm
majorana_LH_0.1_easy
majorana_LH_0.1_ee_momentum
majorana_LH_0.1_e_smallest
majorana_LH_0.1_e_largest
majorana_LH_0.1_ee_theta
majorana_LH_0.1_ee_beam_theta
majorana_LH_0.1_ee_beam_costheta
majorana_LH_0.1_radius_plus
majorana_LH_0.1_distance_between_circles_at_10cm
majorana_LH_0.15_easy
majorana_LH_0.15_ee_momentu